In [21]:
import numpy as np
import pandas as pd
from scipy.sparse.csgraph import connected_components
from math import gcd
from numpy.linalg import eig

In [22]:
df = pd.read_excel('prob.xlsx') 
columns = df.columns[1:]
df[columns] = df[columns].astype(str)

In [23]:
transitions = []
num_semesters = len(columns)

In [29]:
for _, row in df.iterrows():
    for sem in range(num_semesters - 1):
        from_grade = row[columns[sem]]
        to_grade = row[columns[sem + 1]]
        if from_grade != 'F' and to_grade != 'F': 
            transitions.append((from_grade, to_grade))  

transition_counts = pd.DataFrame(transitions, columns=['From', 'To']).value_counts().reset_index()
transition_counts.columns = ['From', 'To', 'Count']

print(transitions)

[('B+', 'B-'), ('B-', 'B'), ('B', 'B'), ('B', 'B+'), ('B+', 'A-'), ('A-', 'A-'), ('A-', 'A'), ('B', 'B'), ('B', 'B+'), ('B+', 'B+'), ('B+', 'B'), ('B', 'B+'), ('B+', 'B'), ('B', 'A+'), ('A', 'A-'), ('A-', 'A'), ('A', 'B+'), ('B+', 'A'), ('A', 'A-'), ('A-', 'A'), ('A', 'A'), ('A', 'A'), ('A', 'A'), ('A', 'A'), ('A', 'A+'), ('A+', 'A'), ('A', 'A+'), ('A+', 'A'), ('B', 'A-'), ('A-', 'A-'), ('A-', 'B+'), ('B+', 'A-'), ('A-', 'A-'), ('A-', 'A-'), ('A-', 'A-'), ('B+', 'B+'), ('B+', 'A-'), ('A-', 'B'), ('B', 'B+'), ('B+', 'A-'), ('A-', 'A-'), ('A-', 'A-'), ('A-', 'A-'), ('A-', 'A'), ('A', 'B+'), ('B+', 'A'), ('A', 'A'), ('A', 'A'), ('A', 'A'), ('B+', 'A-'), ('A-', 'A'), ('A', 'A-'), ('A-', 'A-'), ('A-', 'A-'), ('A-', 'A-'), ('A-', 'A'), ('A-', 'B+'), ('B+', 'A-'), ('A-', 'B+'), ('B+', 'A-'), ('A-', 'A-'), ('A-', 'A-'), ('A-', 'A-'), ('A-', 'A'), ('A', 'B+'), ('B+', 'B'), ('B', 'A'), ('A', 'A'), ('A', 'A'), ('A', 'A'), ('B+', 'B+'), ('B+', 'A-'), ('A-', 'B+'), ('B+', 'A-'), ('A-', 'A-'), ('A-'

In [32]:
grades = sorted(grade for grade in df[columns].stack().unique() if grade != 'F')  
state_space = {grade: i for i, grade in enumerate(grades)}  
P = np.zeros((len(grades), len(grades))) 


['A', 'A+', 'A-', 'B', 'B+', 'B-']


In [26]:
for _, row in transition_counts.iterrows():
    i, j = state_space[row['From']], state_space[row['To']]
    P[i, j] = row['Count']


P = P / P.sum(axis=1, keepdims=True)
print("Transition Probability Matrix (P):")
print(pd.DataFrame(P, index=grades, columns=grades))


Transition Probability Matrix (P):
           A        A+        A-         B        B+        B-
A   0.656716  0.074627  0.164179  0.000000  0.104478  0.000000
A+  0.666667  0.000000  0.166667  0.000000  0.166667  0.000000
A-  0.244755  0.013986  0.496503  0.027972  0.216783  0.000000
B   0.017544  0.035088  0.140351  0.298246  0.473684  0.035088
B+  0.064748  0.000000  0.410072  0.136691  0.366906  0.021583
B-  0.000000  0.000000  0.000000  0.625000  0.250000  0.125000


In [27]:
def check_irreducibility(matrix):
    n_components, _ = connected_components(csgraph=matrix > 0, directed=True, connection='strong')
    return n_components == 1

def check_aperiodicity(matrix):
    periods = []
    for i in range(len(matrix)):
        reachable = np.nonzero(matrix[i, :] > 0)[0]
        if len(reachable) > 0:
            steps = []
            for j in reachable:
                steps.append(gcd(i + 1, j + 1))
            periods.append(gcd(*steps))
    return all(p == 1 for p in periods)

irreducible = check_irreducibility(P)
print("\nIrreducible:", irreducible)
aperiodic = check_aperiodicity(P)
print("Aperiodic:", aperiodic)



Irreducible: True
Aperiodic: True


In [28]:
I = np.identity(len(P))  
A = P.T - I
A_augmented = np.vstack([A, np.ones(len(P))])
b_augmented = np.zeros(len(P))
b_augmented = np.append(b_augmented, 1) 
pi = np.linalg.lstsq(A_augmented, b_augmented, rcond=None)[0]
print("Steady-state probabilities (pi):", pi)

Steady-state probabilities (pi): [0.34118302 0.03228616 0.32564301 0.06470334 0.22796682 0.00821765]
